In [1]:
import os
import yaml
import itertools
from copy import deepcopy
import glob

import numpy as np

In [2]:
# CONFIG_FNAME = "bnfix_minmax_samp_split.yml"
MODEL_NAME = "CellDART"
CONFIG_DIR = "configs/generated_dlpfc"
CONFIG_FNAME_PREFIX = "gen_dlpfc_dlpfc"

In [3]:
config = {
    "lib_params": {},
    "data_params": {},
    "train_params": {},
    "model_params": {},
}

config["data_params"]["all_genes"] = False
config["data_params"]["data_dir"] = "data"
config["data_params"]["dset"] = "dlpfc"
config["data_params"]["samp_split"] = True
config["data_params"]["sc_id"] = "GSE144136"
# config["data_params"]["scaler_name"] = "standard"
config["data_params"]["st_id"] = "spatialLIBD"
config["data_params"]["st_split"] = False

config["model_params"]["celldart_kwargs"] = {}

config["train_params"]["initial_train_epochs"] = 10
config["train_params"]["n_iter"] = 10000
config["train_params"]["reverse_val"] = False
config["train_params"]["pretraining"] = True

if not os.path.exists(os.path.join(CONFIG_DIR, MODEL_NAME)):
    os.makedirs(os.path.join(CONFIG_DIR, MODEL_NAME))

with open(os.path.join(CONFIG_DIR, MODEL_NAME, f"{CONFIG_FNAME_PREFIX}.yml"), "w") as f:
    yaml.safe_dump(config, f)

In [4]:
## CellDART

# data_params
data_params_lists = dict(
    n_markers=[20, 40, 80],
    n_mix=[3, 5, 8],
    n_spots=[5000, 10000, 20000, 50000],
    scaler_name = ["minmax", "standard"],
)
# model_params
model_params_lists = dict(
    bn_momentum = [0.01, 0.1, 0.9, 0.99],
    emb_dim = [32, 64],
)


# train_params
train_params_lists = dict(
    alpha =  [0.1, 0.6, 1.0, 2.0],
    alpha_lr = [1, 2, 5, 10],
    batch_size = [256, 512, 1024],
    lr =[0.01, 0.001, 0.0001],
)


In [5]:
total_configs = 1
for value in data_params_lists.values():
    total_configs *= len(value)
for value in model_params_lists.values():
    total_configs *= len(value)
for value in train_params_lists.values():
    total_configs *= len(value)
total_configs

82944

In [6]:
config

{'lib_params': {},
 'data_params': {'all_genes': False,
  'data_dir': 'data',
  'dset': 'dlpfc',
  'samp_split': True,
  'sc_id': 'GSE144136',
  'st_id': 'spatialLIBD',
  'st_split': False},
 'train_params': {'initial_train_epochs': 10,
  'n_iter': 10000,
  'reverse_val': False,
  'pretraining': True},
 'model_params': {'celldart_kwargs': {}}}

In [7]:
rng = np.random.default_rng(45836)

yes_samples = set(rng.choice(total_configs, size=1000, replace=False))


data_params_l = []
for kv_tuples in itertools.product(
    *[[(k, v) for v in vlist] for k, vlist in data_params_lists.items()]
):
    data_params_l.append(dict(kv_tuples))

model_params_l = []
for kv_tuples in itertools.product(
    *[[(k, v) for v in vlist] for k, vlist in model_params_lists.items()]
):
    model_params_l.append(dict(kv_tuples))

train_params_l = []
for kv_tuples in itertools.product(
    *[[(k, v) for v in vlist] for k, vlist in train_params_lists.items()]
):
    train_params_l.append(dict(kv_tuples))

for i, (data_params, model_params, train_params) in enumerate(
    itertools.product(data_params_l, model_params_l, train_params_l)
):
    if i not in yes_samples:
        continue

    new_config = deepcopy(config)
    
    new_config["data_params"].update(data_params)
    new_config["model_params"]["celldart_kwargs"].update(model_params)
    new_config["train_params"].update(train_params)
    
    new_config["lib_params"]["manual_seed"] = int(rng.integers(0, 2**32))

    version = f"{CONFIG_FNAME_PREFIX}-{i}"
    new_config["model_params"]["model_version"] = version

    with open(os.path.join(CONFIG_DIR, MODEL_NAME, f"{version}.yml"), "w") as f:
        yaml.safe_dump(new_config, f)

In [8]:
print(yaml.safe_dump(new_config))


data_params:
  all_genes: false
  data_dir: data
  dset: dlpfc
  n_markers: 80
  n_mix: 8
  n_spots: 50000
  samp_split: true
  sc_id: GSE144136
  scaler_name: standard
  st_id: spatialLIBD
  st_split: false
lib_params:
  manual_seed: 1702444108
model_params:
  celldart_kwargs:
    bn_momentum: 0.99
    emb_dim: 32
  model_version: gen_dlpfc_dlpfc-82672
train_params:
  alpha: 0.1
  alpha_lr: 2
  batch_size: 1024
  initial_train_epochs: 10
  lr: 0.001
  n_iter: 10000
  pretraining: true
  reverse_val: false



In [9]:
lines = [
    os.path.basename(name)
    for name in sorted(glob.glob(os.path.join(CONFIG_DIR, MODEL_NAME, f"{CONFIG_FNAME_PREFIX}-*.yml")))
]
with open(
    os.path.join(CONFIG_DIR, MODEL_NAME, "a_list.txt"),
    mode="wt",
    encoding="utf-8",
) as myfile:
    myfile.write("\n".join(lines))
    myfile.write("\n")